In [3]:
path = "~/ml/models/keras/resnet152/resnet152_weights_tf.h5"

In [4]:
from resnet import resnet152

Using TensorFlow backend.


In [6]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.callbacks import TensorBoard, ModelCheckpoint
import os

# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = '/home/csweeney/AttentionSceneClassification/images/'
validation_data_dir = '/home/csweeney/AttentionSceneClassification/images_test/'
nb_train_samples = 1000
nb_validation_samples = 800
epochs = 50
batch_size = 4

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

WEIGHTS_RESNET = os.path.expanduser("~/ml/models/keras/resnet152/resnet152_weights_tf.h5")


# Load Resnet 152 model and construct feature extraction submodel
resnet_model = resnet152.resnet152_model(WEIGHTS_RESNET)
print resnet_model.summary()
top_model = Sequential()
top_model.add(Dense(67, activation='softmax',input_shape=resnet_model.output_shape[1:]))


new_model = Sequential() #new model
for layer in resnet_model.layers: 
    new_model.add(layer)

new_model.add(top_model)
print new_model.summary()

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
# for layer in new_model.layers[:25]:
#     layer.trainable = False

# feature_layer = 'avg_pool'
# feature_vector_size = int(resnet_model.get_layer(feature_layer).output.shape[-1])
# features_model = Model(inputs=resnet_model.input,
#                        outputs=resnet_model.get_layer(feature_layer).output)
tb = TensorBoard(log_dir='./logs', write_images=True)
mc = ModelCheckpoint(filepath="weights.best.hdf5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')
new_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])
    
new_model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    nb_epoch=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples,
    callbacks=[tb,mc])
# fit_generator(self, generator, samples_per_epoch, nb_epoch, verbose=1, callbacks=None, validation_data=None, nb_val_samples=None, class_weight=None, max_q_size=10, nb_worker=1, pickle_safe=False, initial_epoch=0)

Found 14280 images belonging to 67 classes.
Found 1347 images belonging to 67 classes.
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
data (InputLayer)                (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv1_zeropadding (ZeroPadding2D (None, 230, 230, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9408                                         
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256                                          
____

____________________________________________________________________________________________________
res4b35_branch2a_relu (Activatio (None, 14, 14, 256)   0                                            
____________________________________________________________________________________________________
res4b35_branch2b_zeropadding (Ze (None, 16, 16, 256)   0                                            
____________________________________________________________________________________________________
res4b35_branch2b (Conv2D)        (None, 14, 14, 256)   589824                                       
____________________________________________________________________________________________________
bn4b35_branch2b (BatchNormalizat (None, 14, 14, 256)   1024                                         
____________________________________________________________________________________________________
scale4b35_branch2b (Scale)       (None, 14, 14, 256)   512                                 

ValueError: Input 0 is incompatible with layer res2a_branch1: expected axis -1 of input shape to have value 64 but got shape (None, 55, 55, 256)